In [1]:
import pandas as pd
import numpy as np
import datetime

import xgboost as xgb
import sklearn

from sklearn.model_selection import train_test_split

%matplotlib inline

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [72]:
train = pd.read_csv('train.csv', '\t', index_col='Unnamed: 0')
test = pd.read_csv('test.csv', '\t', index_col='Unnamed: 0')
X = train.drop('0', axis=1).as_matrix()
y = train['0']
X_test = test.drop('0', axis=1).as_matrix()
X.shape

(30500, 345)

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.33, random_state=42)

In [40]:
import xgboost as xgb

In [41]:
bst = xgb.XGBClassifier()

In [57]:
dtrain = xgb.DMatrix(X_train, y_train)
xgtrain = xgb.DMatrix(X_train)
dvalidate = xgb.DMatrix(X_validate)

D = xgb.DMatrix(X, y)
X_vals = xgb.DMatrix(X)
dtest = xgb.DMatrix(X_test)

In [75]:
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
for l in learning_rate:
    param = {'max_depth':3, 'min_child_weight':2.4, 'subsample':0.7,
                 'colsample_bytree':0.7, 'eta':l, 'silent':1, 'objective':'binary:logistic' }
    num_rounds = 120
    bst = xgb.train(param, dtrain, num_rounds)
    AUC_train = roc_auc_score(y_train, bst.predict(xgtrain))
    AUC_validate = roc_auc_score(y_validate, bst.predict(dvalidate))
    print('%.6f %.6f lrate = %f' %  (AUC_train, AUC_validate, l))

0.689202 0.684542 lrate = 0.000100
0.690983 0.686101 lrate = 0.001000
0.705088 0.695048 lrate = 0.010000
0.785006 0.726178 lrate = 0.100000
0.819024 0.720817 lrate = 0.200000
0.840513 0.713943 lrate = 0.300000


In [77]:
param = {'max_depth':3, 'min_child_weight':2.4, 'subsample':0.7, 'colsample_bytree':0.7,
         'eta':0.1, 'silent':1, 'objective':'binary:logistic' }
num_round = 120
xgb_best = xgb.train(param, dtrain, num_round)

In [59]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_scores = xgb_best.predict(dvalidate)
roc_auc_score(y_validate, y_scores)

0.72164658892868883

In [79]:
xgb_best = xgb.train(param, D, num_round)
y_res = xgb_best.predict(dtest)

In [80]:
#Сохраняем результат в csv
test_pd = pd.DataFrame(y_res,columns=['_VAL_'])
test_pd.index.names = ['_ID_']
test_pd.to_csv('xgb.csv')
print(test_pd.shape)
test_pd.head(2)

(4166, 1)


,_VAL_
_ID_,
0,0.123826
1,0.385335


In [47]:
y_res

array([ 0.16752338,  0.14745398,  0.10030134, ...,  0.27551308,
        0.11048718,  0.13620201], dtype=float32)

In [62]:
range(1, 300, 25)

[1, 26, 51, 76, 101, 126, 151, 176, 201, 226, 251, 276]